In [ ]:
# Imports
import os
import re
import traceback
import json
import nix_utils, llm_prompts, llm_client
import requests
import subprocess
from typing import Optional, Union, List
from pathlib import Path
from pydantic import BaseModel

### Prepare a new Git repository

In [ ]:
dirname = Path("output")

In [ ]:
dirname.mkdir(parents=True, exist_ok=True)

In [ ]:
# Make Git ignore the global configuration files
os.environ['GIT_CONFIG_SYSTEM'] = "/dev/null"
os.environ['GIT_CONFIG_GLOBAL'] = "/dev/null"

In [ ]:
subprocess.run(("git", "init", "--initial-branch=main"), cwd=dirname, check=True)

In [ ]:
_ = subprocess.run(("git", "config", "user.name", "AI-Nix-Packager"), cwd=dirname, check=True)
_ = subprocess.run(("git", "config", "user.email", ""), cwd=dirname, check=True)

### Initialize a new Git branch

In [ ]:
# TODO: don't hardcode
#project_name = "findtow" # https://git.sr.ht/~martijnbraam/findtow
project_name = "gResistor" # https://github.com/stethewwolf/gResistor
#project_name = "postmarketos-tweaks" # https://gitlab.postmarketos.org/postmarketOS/postmarketos-tweaks

In [ ]:
_ = subprocess.run(("git", "checkout", "--orphan", project_name), cwd=dirname, check=True)
_ = subprocess.run(("git", "rm", "-rf", "."), cwd=dirname, check=False) # Ignore failures that happen when there are no files

### Generate the default.nix file using nix-init

In [ ]:
# Now please run nix-init inside of the output directory
nix_file = dirname / "default.nix"
if not os.path.isfile(nix_file):
    if not os.path.isfile("default.nix"):
        raise Exception("default.nix not found, please run nix-init inside of the output directory.")
    else:
        raise Exception("Make sure to run nix-init inside of the output directory, not inside the root of the project!")

In [ ]:
def git_commit(message):
    subprocess.run(("git", "add", "--all"), cwd=dirname, check=True)
    subprocess.run(("git", "commit", "-m", message), cwd=dirname, check=True)

In [ ]:
git_commit("Run nix-init")

In [ ]:
# TODO: run nix-format after every step

In [ ]:
# TODO: pin Nixpkgs revision to the one used by this flake
def nix_build():
    return subprocess.run(
        (
            "nix-build",
            "--no-out-link",
            "--log-format", "internal-json",
             "-E", "(import <nixpkgs> { }).callPackage ./. { }"
        ),
        capture_output=True,
        encoding='utf-8',
        cwd=dirname,
        check=False,
        env={
            "PATH": os.getenv("PATH"),
            "NIX_PATH": os.getenv("NIX_PATH"),
            "NO_COLOR": "", # TODO: debug Nix why this has no effect when --log-format is internal-json
            "NIXPKGS_ALLOW_UNFREE": "1",
        },
    )

In [ ]:
build_result = nix_build()

In [ ]:
# Analyze and filter output

if build_result.returncode == 0:
    print("The nix-build command succeeded 🎉")
else:
    print("The nix-build command failed, let's see what the error was.")
    print(build_result.stdout)
    error_message = utils.parse_error_from_nix_output(build_result.stderr)
    print(error_message)

### Run the LLM-Debug Loop by providing the default.nix and error.log file

In [ ]:
# TODO: find instances of the absolute path to the current directory and rewrite them
# TODO: special-case errors like evaluation aborted with the following error message:
# 'lib.customisation.callPackageWith: Function called without required argument "libblkid" at /home/u/Dokumente/repos/RAG-with-Langchain/output/default.nix:8'
# to provide some help to the LLM
# TODO: validate syntax and help the model if it is wrong
# TODO: detect if the model ever outputs the same thing twice and tell the model
# TODO: detect if the error message is the same as before and tell the model that the suggestion did not fix the error
# TODO: take existing derivations from Nixpkgs and modify them, e.g. by deleting lines. Then fine-tune an LLM with the error messages
# TODO: prompt the model to tell if thinks that a dependency is missing and then use something like search.nixos.org but lokal (maybe nix-index) to search for it, then tell the model which dependency to add
# TODO: Run multiple different models at each step and compare
# TODO: detect changes that the LLM should not make, such as changing the hash and revert them
# TODO: if the model does not support reasoning, first ask the model for suggestions on fixing the error, then asking it to implement those suggestions

In [ ]:
print("Dependencies loaded successfully!")

In [ ]:
def debug_loop(initial_nix_path: str, max_attempts: int = 1):
    attempts = 0
    build_succeeded = False
    error_message = ""

    last_error = None
    last_fix_code = None
    error_history = []
    dirname= Path("output/")
    while not build_succeeded and attempts < max_attempts:
        print(f"\n--- Attempt {attempts + 1}/{max_attempts} ---")

        try:
            default_nix_content = nix_utils.read_file_content(initial_nix_path)
            build_result = nix_utils.nix_build(dirname)

            if build_result.returncode == 0:
                print("The nix-build command succeeded 🎉")
            else:
                print("The nix-build command failed, let's see what the error was.")
                print(build_result.stdout)
                error_message = nix_utils.parse_error_from_nix_output(build_result.stderr)
                print(error_message)
                # Write the error message to the log file as requested
                error_log_path = os.path.join(dirname, "error.log")
                nix_utils.write_file_content(error_log_path, error_message)
                
                print("Error message parsed and saved to error.log.")

            # Detect hallucination loop
            if error_message == last_error and last_fix_code is not None:
                print("⚠️  Same error as last attempt detected.")

            print("\n--- Getting LLM Fix ---")
            fix_response = llm_client.get_llm_fix(default_nix_content, error_message, error_history)

            if fix_response:
                if error_message == last_error and fix_response.corrected_nix_code == last_fix_code:
                    print("🚫 Same error and same fix as last attempt detected. Stopping to avoid hallucination loop.")
                    break

                # Apply fix
                print("\n--- Applying LLM Fix ---")
                print("Explanation:\n", fix_response.explanation)
                nix_utils.write_file_content(initial_nix_path, fix_response.corrected_nix_code)

                # Save attempt in history
                error_history.append((error_message, fix_response.corrected_nix_code))
                last_error = error_message
                last_fix_code = fix_response.corrected_nix_code

            else:
                print("Could not get a fix from the LLM. Stopping.")
                break

        except FileNotFoundError as e:
            print(f"Fatal error: {e}. Please ensure the initial Nix file exists.")
            break
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            print(traceback.format_exc())
            break

        attempts += 1

    if not build_succeeded:
        print("\n--- Final Status ---")
        print(f"Failed to fix the Nix file after {max_attempts} attempts.")
        print("Please review the 'default.nix' and 'error.log' for more details.")


# --- Call the main function with a mock setup ---
if __name__ == "__main__":
    # Create a directory for the output files
    output_dir = "output"
    os.makedirs(output_dir, exist_ok=True)
    
    # Start the debugging loop
    debug_loop("output/default.nix")